## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/Weather_database.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,2021-07-20 01:50:21,42.0970,-79.2353,71.24,78,1,6.91,clear sky
1,1,Mashhad,IR,2021-07-20 01:50:46,36.2970,59.6062,71.74,35,0,4.61,clear sky
2,2,Gao,ML,2021-07-20 01:50:46,16.6362,1.6370,89.02,35,100,16.35,overcast clouds
3,3,Newberry,US,2021-07-20 01:50:46,34.2746,-81.6187,74.14,97,100,3.38,light rain
4,4,Surdulica,RS,2021-07-20 01:50:47,42.6906,22.1708,67.69,99,90,3.00,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,2021-07-20 01:50:21,42.0970,-79.2353,71.24,78,1,6.91,clear sky
1,1,Mashhad,IR,2021-07-20 01:50:46,36.2970,59.6062,71.74,35,0,4.61,clear sky
3,3,Newberry,US,2021-07-20 01:50:46,34.2746,-81.6187,74.14,97,100,3.38,light rain
4,4,Surdulica,RS,2021-07-20 01:50:47,42.6906,22.1708,67.69,99,90,3.00,overcast clouds
10,10,Vacha,RU,2021-07-20 01:50:49,55.8008,42.7706,63.81,95,57,4.50,broken clouds
12,12,Saint-Philippe,RE,2021-07-20 01:50:50,-21.3585,55.7679,69.39,74,69,15.37,broken clouds
13,13,Arraial Do Cabo,BR,2021-07-20 01:50:50,-22.9661,-42.0278,66.13,58,25,27.07,scattered clouds
16,16,Margate,GB,2021-07-20 01:47:28,51.3813,1.3862,64.96,93,5,8.19,clear sky
18,18,Lourdes,FR,2021-07-20 01:50:51,43.1000,-0.0500,66.94,94,20,5.75,few clouds
19,19,Camacha,PT,2021-07-20 01:50:51,33.0833,-16.3333,68.54,68,75,8.05,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                278
City                   278
Country                278
Date                   278
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
#preferred_cities_df.head()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,71.24,clear sky,42.0970,-79.2353,
1,Mashhad,IR,71.74,clear sky,36.2970,59.6062,
3,Newberry,US,74.14,light rain,34.2746,-81.6187,
4,Surdulica,RS,67.69,overcast clouds,42.6906,22.1708,
10,Vacha,RU,63.81,broken clouds,55.8008,42.7706,
12,Saint-Philippe,RE,69.39,broken clouds,-21.3585,55.7679,
13,Arraial Do Cabo,BR,66.13,scattered clouds,-22.9661,-42.0278,
16,Margate,GB,64.96,clear sky,51.3813,1.3862,
18,Lourdes,FR,66.94,few clouds,43.1000,-0.0500,
19,Camacha,PT,68.54,broken clouds,33.0833,-16.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,71.24,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,Mashhad,IR,71.74,clear sky,36.2970,59.6062,Khanye Sabz Hotel
3,Newberry,US,74.14,light rain,34.2746,-81.6187,Hampton Inn Newberry-Opera House
4,Surdulica,RS,67.69,overcast clouds,42.6906,22.1708,Apartmani Park 2016
10,Vacha,RU,63.81,broken clouds,55.8008,42.7706,Dom Druga
12,Saint-Philippe,RE,69.39,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Arraial Do Cabo,BR,66.13,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
16,Margate,GB,64.96,clear sky,51.3813,1.3862,Premier Inn Margate hotel
18,Lourdes,FR,66.94,few clouds,43.1000,-0.0500,"Hôtel Saint Etienne, Lourdes"
19,Camacha,PT,68.54,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))